In [2]:
import random
import time
import csv

class HexGame:
    def __init__(self, size):
        self.size = size
        self.board = [['-' for _ in range(size)] for _ in range(size)]
        self.total_search_space = 0
        self.total_elapsed_time = 0

    def print_board(self):
        for row in self.board:
            print(' '.join(row))
        print()

    def is_valid_move(self, move):
        x, y = move
        return 0 <= x < self.size and 0 <= y < self.size and self.board[x][y] == '-'

    def is_winner(self, player):
        # Check if there's a winning path from top to bottom (for 'X') or from left to right (for '#')
        if player == '#':
            return any(self.board[0][j] == '#' and self.dfs((0, j), '#', set()) for j in range(self.size))
        else:
            return any(self.board[i][0] == 'X' and self.dfs((i, 0), 'X', set()) for i in range(self.size))

    def dfs(self, pos, player, visited):
        if pos in visited or not self.is_valid_move(pos) or self.board[pos[0]][pos[1]] != player:
            return False
        if (player == '#' and pos[0] == self.size - 1) or (player == 'X' and pos[1] == self.size - 1):
            return True

        visited.add(pos)
        neighbors = [(pos[0] + 1, pos[1]), (pos[0] - 1, pos[1]), (pos[0], pos[1] + 1), (pos[0], pos[1] - 1)]
        return any(self.dfs(n, player, visited.copy()) for n in neighbors)

    def get_empty_cells(self):
        return [(i, j) for i in range(self.size) for j in range(self.size) if self.board[i][j] == '-']

    def is_game_over(self):
        return self.is_winner('#') or self.is_winner('X') or len(self.get_empty_cells()) == 0

    def minimax(self, depth, maximizing_player):
        if depth == 0 or self.is_game_over():
            if self.is_winner('#'):
                return -1
            elif self.is_winner('X'):
                return 1
            else:
                return 0

        if maximizing_player:
            max_eval = float('-inf')
            for move in self.get_empty_cells():
                self.board[move[0]][move[1]] = 'X'
                self.total_search_space += 1
                start_time = time.time()
                eval = self.minimax(depth - 1, False)
                elapsed_time = time.time() - start_time
                self.total_elapsed_time += elapsed_time
                self.board[move[0]][move[1]] = '-'
                max_eval = max(max_eval, eval)
            return max_eval
        else:
            min_eval = float('inf')
            for move in self.get_empty_cells():
                self.board[move[0]][move[1]] = '#'
                self.total_search_space += 1
                start_time = time.time()
                eval = self.minimax(depth - 1, True)
                elapsed_time = time.time() - start_time
                self.total_elapsed_time += elapsed_time
                self.board[move[0]][move[1]] = '-'
                min_eval = min(min_eval, eval)
            return min_eval

    def get_best_move(self):
        best_moves = []
        best_eval = float('-inf')
        for move in self.get_empty_cells():
            self.board[move[0]][move[1]] = 'X'
            self.total_search_space += 1
            start_time = time.time()
            eval = self.minimax(3, True)  # Depth of the search tree
            elapsed_time = time.time() - start_time
            self.total_elapsed_time += elapsed_time
            self.board[move[0]][move[1]] = '-'
            if eval > best_eval:
                best_eval = eval
                best_moves = [move]
            elif eval == best_eval:
                best_moves.append(move)
        return random.choice(best_moves)

def play_hex(size):
    game = HexGame(size)
    maximizing_player = 'X'
    minimizing_player = '#'
    print("Maximizing Player: X")
    print("Minimizing Player: #\n")
    print("Initial Board:\n")

    while not game.is_game_over():
        game.print_board()
        if maximizing_player == 'X':
            move = game.get_best_move()
            print("X's move:", move)
            game.board[move[0]][move[1]] = 'X'
            maximizing_player, minimizing_player = minimizing_player, maximizing_player
        else:
            move = game.get_best_move()
            print("#'s move:", move)
            game.board[move[0]][move[1]] = '#'
            maximizing_player, minimizing_player = minimizing_player, maximizing_player
    game.print_board()
    if game.is_winner('#'):
        print("# wins!")
    elif game.is_winner('X'):
        print("X wins!")
    else:
        print("It's a draw!")
    print("Total Search Space:", game.total_search_space)
    print("Total Elapsed Time:", game.total_elapsed_time, "seconds")
    algorithm_name = f'MiniMax {size} X {size}'
    search_space_size = game.total_search_space
    elapsed_time = game.total_elapsed_time
    append_to_csv(algorithm_name, search_space_size, elapsed_time)
    
def append_to_csv(algorithm_name, search_space_size, elapsed_time):
    with open('hex_results.csv', 'w', newline='') as csvfile:
        fieldnames = ['Algorithm Name', 'Search Space Size', 'Elapsed Time']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        # Write header if file is empty
        if csvfile.tell() == 0:
            writer.writeheader()

        writer.writerow({'Algorithm Name': algorithm_name, 'Search Space Size': search_space_size, 'Elapsed Time': elapsed_time})

if __name__ == "__main__":
    size = int(input("Enter the size of the HEX grid: "))
    play_hex(size)


Enter the size of the HEX grid: 6
Maximizing Player: X
Minimizing Player: #

Initial Board:

- - - - - -
- - - - - -
- - - - - -
- - - - - -
- - - - - -
- - - - - -

X's move: (4, 3)
- - - - - -
- - - - - -
- - - - - -
- - - - - -
- - - X - -
- - - - - -

#'s move: (2, 1)
- - - - - -
- - - - - -
- # - - - -
- - - - - -
- - - X - -
- - - - - -

X's move: (1, 3)
- - - - - -
- - - X - -
- # - - - -
- - - - - -
- - - X - -
- - - - - -

#'s move: (0, 5)
- - - - - #
- - - X - -
- # - - - -
- - - - - -
- - - X - -
- - - - - -

X's move: (2, 5)
- - - - - #
- - - X - -
- # - - - X
- - - - - -
- - - X - -
- - - - - -

#'s move: (4, 0)
- - - - - #
- - - X - -
- # - - - X
- - - - - -
# - - X - -
- - - - - -

X's move: (1, 1)
- - - - - #
- X - X - -
- # - - - X
- - - - - -
# - - X - -
- - - - - -

#'s move: (3, 5)
- - - - - #
- X - X - -
- # - - - X
- - - - - #
# - - X - -
- - - - - -

X's move: (1, 0)
- - - - - #
X X - X - -
- # - - - X
- - - - - #
# - - X - -
- - - - - -

#'s move: (0, 2)
- - # -